In [41]:
import pandas as pd
import nltk

nltk.download("punkt")
from nltk.tokenize import sent_tokenize


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [42]:
English_Text = pd.read_csv(
    "/content/drive/MyDrive/Colab Notebooks/Data_Sentiment/Reviews.csv",
    encoding="cp1256"
)


In [43]:
English_Text.sample(5)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
260507,260508,B000NMJWZO,AQ6SHOW0VMZGF,"K. Padgett ""familyof5""",1,1,5,1348012800,Impressed,I hesitated for months before purchasing Pamel...
412234,412235,B000SATIMI,A1B5M4P81KW8TV,"Kaye ""Kaye""",0,0,5,1335052800,Fresh and great tasting!!!,I love this tea! It had fresh dried berries al...
19070,19071,B00375LB6W,A30NZHE1H756IU,RoseMumof3,0,0,5,1338768000,Perfect for our cupcakes!,We chose a Thing 1/Thing 2 theme for our twins...
288681,288682,B001ELL71K,A1Z4LX3Q1YMF84,"Cindy A. Osterhout ""Cindy O""",0,0,5,1349740800,AMAZINGLY GOOD PUMPKIN SPICE BREAD MIX,This is the BEST Pumpkin Spice Bread Mix that ...
103708,103709,B000Q6XR0S,A1KEK09ZA6J9P8,Colleen M. Schneider,1,1,5,1342742400,So good!...........,One of my carry-over faves from childhood is a...


In [44]:
print(English_Text.shape)
print(English_Text.columns)

(568454, 10)
Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')


In [45]:
def score_to_label(score):
    if score <= 2:
        return "Negative"
    elif score == 3:
        return "Neutral"
    else:
        return "Positive"

English_Text["Label"] = English_Text["Score"].apply(score_to_label)


In [46]:
English_Text["Label"].value_counts()


,count
Label,
Positive,443777
Negative,82037
Neutral,42640


In [47]:
label_map = {"Negative": 0, "Neutral": 1, "Positive": 2}
English_Text["Label"] = English_Text["Label"].map(label_map)


In [48]:
English_Text["Label"].value_counts()


,count
Label,
2,443777
0,82037
1,42640


In [49]:
English_Text["Text"] = (English_Text["Summary"].fillna("").str.strip()+ " [SEP] "+ English_Text["Text"].fillna("").str.strip())
# Combine review summary and full text into a single field using [SEP] as a separator
# This provides richer context for sentiment analysis and prevents loss of information

In [50]:
English_Text[["Text", "Label"]].sample(5)


,Text,Label
20255,Otis will eat it! [SEP] Otis is what I call a ...,2
188977,Should be called Natural Drano [SEP] I take th...,2
566203,"Diabetics, Beware: FANTASTIC except for the ad...",2
340439,The real deal [SEP] I recently went on a study...,2
27736,"Fantastic! [SEP] This tea is amazing, I cannot...",2


In [51]:
print(English_Text['Text'].isnull().sum())
print(English_Text.duplicated(subset=["Text"]).sum())



0
173484


In [52]:
English_Text = English_Text.dropna(subset=["Text"])
English_Text = English_Text.drop_duplicates(subset=["Text"])


In [53]:
print(English_Text['Text'].isnull().sum())
print(English_Text.duplicated(subset=["Text"]).sum())
print(English_Text.shape)

0
0
(394970, 11)


In [54]:
English_Text.rename(columns={"Text": "text", "Label": "label"},inplace=True)


In [55]:
# English_Final = pd.DataFrame(records)


In [56]:
print(English_Text.duplicated(subset=["text"]).sum())
English_Text = English_Text.drop_duplicates(subset=["text"])

0


In [57]:
English_Text.shape


(394970, 2)

In [58]:
English_Text["label"].value_counts()

,count
label,
2,307787
0,57346
1,29837


In [59]:
samples_per_label = {
    1: 29837,   # Neutral
    0: 40000,   # Negative
    2: 80163    # Positive
}

parts = []

for label, n in samples_per_label.items():
    part = English_Text[English_Text["label"] == label].sample(n=n,random_state=42)
    parts.append(part)

English_150k = (pd.concat(parts).sample(frac=1, random_state=42).reset_index(drop=True))

print(English_150k["label"].value_counts())
print(English_150k.shape)


label
2    80163
0    40000
1    29837
Name: count, dtype: int64
(150000, 2)


In [60]:
English_150k.to_csv(
    "english_sentiment_150k_balanced.csv",
    index=False,
    encoding="utf-8-sig"
)

print("English balanced dataset saved ✅")


English balanced dataset saved ✅
